## Requisitos

In [1]:
import pandas as pd
import numpy as np
import neural_net as nn
import math

## Funções Auxiliares

In [2]:
# Geração de conjuntos estratificados para validação cruzada
def k_folds(df, target_attr, k=5, shuffle_seed=99):
    # Número de classes (valores distintos na coluna alvo)
    unique_classes = df[target_attr].unique()
    # Separa o dataset em 2 de acordo com o valor do atributo alvo e embaralha as entradas dentro de cada subset
    data_by_class = [df.loc[df[target_attr] == c].sample(frac=1, random_state=shuffle_seed) for c in unique_classes]
    # Cria uma lista com k dataframes
    folds = [pd.DataFrame() for i in range(k)]
    # Divide os dados em k dataframes de forma estratificada
    for class_data in data_by_class:
        n_rows = class_data.iloc[:, -1].count()
        fold_size = math.ceil(n_rows / k)
        for i in range(k):
            folds[i] = folds[i].append(class_data.iloc[i*fold_size:(i+1)*fold_size])
    # Embaralha as instâncias dentro de cada fold
    for i in range(len(folds)): folds[i] = folds[i].sample(frac=1, random_state=shuffle_seed)
    return folds

In [3]:
# Dada uma lista, retorna a lista sem o elemento de index i
def remove_index(list, index):
    if len(list) - 1 > index:
        return list[:index] + list[(index+1):]
    elif len(list) - 1 == index and index >= 0:
        return list[:index]
    else: raise Exception('index inválido')

In [4]:
# Retorna uma tupla que lista os atributos preditivos no formato (attr_name, attr_type, [possible_values])
# a partir de um arquivo csv "file" com separador "sep" no formato 
# "attr1<sep>attr2  
# attr1_type<sep>attr2_type
# value1<sep>value2"
def read_attr_list(file, sep):
  attr_df = pd.read_csv(file, sep=sep)

  attr_list = []

  for col in range(len(attr_df.columns)):
    attr_tuple = (attr_df.columns[col], attr_df.iloc[0, col])
    if len(list(attr_df[attr_df.columns[col]].dropna().drop([0]))) != 0:
      attr_tuple += (list(attr_df[attr_df.columns[col]].dropna().drop([0])),)
    attr_list.append(attr_tuple)

  return attr_list

In [5]:
# Treina uma rede neural, testa um conjunto de teste e retorna a acurácia
def test_neural_net(training_df, testing_df, attr_list, target_attr="target", alpha=0.05, lamb=0.0, intermediate_net=[2], n_outputs=1):
    correct_predictions = 0
    incorrect_predictions = 0

    network = [len(attr_list), *intermediate_net, n_outputs]
    print(network)

    net = nn.NeuralNetwork(network, alpha=0.05, lamb=0.0)

    # TODO
    inputs = []
    exp_outputs = []

    net.train(inputs, exp_outputs)

    for idx, instance in testing_df.iterrows():
      predicted_class = net.classify(instance_in, instance_out, n_outputs)
      actual_class = testing_df.at[idx, target_attr]
      if actual_class == predicted_class:
        correct_predictions += 1
      else:
        incorrect_predictions += 1
    accuracy = round(correct_predictions / testing_df.shape[0], 2)
    return accuracy

## Dataset 1 - house-votes-84.tsv

In [6]:
df = pd.read_csv('datasets/house-votes-84.tsv', sep='\t')

target_attr = "target"
n_folds = 5

# Gera conjuntos estratificados para a validação cruzada
folds = k_folds(df, target_attr, n_folds)
attr_list = read_attr_list("datasets/house-votes-attr.csv", ";")
folds_accuracy = []

# Teste da rede neural para cada um dos folds
for i in range(len(folds)):
  training_df = pd.concat(remove_index(folds, i))
  testing_df = folds[i]

  accuracy = test_neural_net(training_df, testing_df, attr_list, target_attr, alpha=0.05, lamb=0.0, intermediate_net=[8])
  folds_accuracy.append([accuracy])

# Agregação dos Resultados
stats = pd.DataFrame(folds_accuracy, columns=["Accuracy"])
desc = stats.describe()
results = pd.concat([stats, desc.loc[['mean', 'std']] ])
results

[16, 8, 1]


ValueError: operands could not be broadcast together with shapes (0,) (8,17) 